# Gemini Function Calling - 구축 중심 실습

**학습 목표**
- 함수 스키마(FunctionDeclaration) 직접 설계
- 다중 함수 시스템 구축
- 순차 호출 파이프라인 구현

**시나리오: 스마트홈 제어 시스템**

---
## 1. 환경 설정

In [ ]:
import os
from dotenv import load_dotenv
from google import genai
from google.genai import types

load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
print("✅ Client 연결 완료")

---
## 2. 단일 함수 구축

### 2.1 Python 함수 정의

Function Calling에서 **docstring**이 매우 중요합니다.  
SDK가 docstring을 파싱하여 자동으로 스키마를 생성합니다.

**좋은 docstring의 조건:**
- 함수의 목적을 명확히 설명
- 각 파라미터의 타입과 의미 명시
- 반환값 설명

In [ ]:
# 예시: 잘 작성된 함수 정의 (제공됨)
def get_room_temperature(room: str) -> dict:
    """특정 방의 현재 온도를 조회합니다.
    
    Args:
        room: 방 이름. 가능한 값: "거실", "안방", "주방", "욕실"
    
    Returns:
        현재 온도와 습도를 포함한 딕셔너리
    """
    temperatures = {
        "거실": {"temp": 22, "humidity": 45},
        "안방": {"temp": 20, "humidity": 50},
        "주방": {"temp": 24, "humidity": 55},
        "욕실": {"temp": 26, "humidity": 70},
    }
    return temperatures.get(room, {"temp": 20, "humidity": 50})

# 테스트
print(get_room_temperature("거실"))

In [ ]:
# TODO(human): 조명 제어 함수 정의
# 목적: Function Calling에 적합한 함수 설계 연습
# 힌트:
#   - 함수명: set_light
#   - 파라미터: room(str), brightness(int 0~100), color_temp(str: "warm"/"cool"/"daylight")
#   - 반환: {"room": ..., "brightness": ..., "color_temp": ..., "status": "설정 완료"}
#   - docstring 필수!
# 예상: 모델이 "거실 조명 50%로 따뜻하게 설정해줘" 같은 요청 처리 가능

def set_light(room: str, brightness: int, color_temp: str) -> dict:
    # TODO(human): docstring 작성
    # TODO(human): 함수 구현
    pass

### 2.2 FunctionDeclaration 스키마 작성

Python 함수를 직접 전달하는 대신, JSON 스키마로 명시적으로 정의할 수 있습니다.  
이 방식은 다른 언어에서도 동일하게 사용 가능하고, 더 세밀한 제어가 가능합니다.

**스키마 구조:**
```python
{
    "name": "함수명",
    "description": "함수 설명",
    "parameters": {
        "type": "object",
        "properties": {
            "param1": {"type": "string", "description": "설명"},
            "param2": {"type": "integer", "description": "설명"},
        },
        "required": ["param1"]
    }
}
```

In [ ]:
# 예시: FunctionDeclaration 스키마 (제공됨)
get_room_temperature_declaration = {
    "name": "get_room_temperature",
    "description": "특정 방의 현재 온도를 조회합니다.",
    "parameters": {
        "type": "object",
        "properties": {
            "room": {
                "type": "string",
                "description": "방 이름. 가능한 값: 거실, 안방, 주방, 욕실",
                "enum": ["거실", "안방", "주방", "욕실"]
            }
        },
        "required": ["room"]
    }
}

print("📋 스키마 정의 완료")
print(get_room_temperature_declaration)

In [ ]:
# TODO(human): set_light 함수의 FunctionDeclaration 스키마 작성
# 목적: JSON 스키마 형식 이해
# 힌트:
#   - brightness: type="integer", minimum=0, maximum=100
#   - color_temp: type="string", enum=["warm", "cool", "daylight"]
#   - 모든 파라미터 required
# 예상: 스키마가 위의 get_room_temperature_declaration과 유사한 구조

set_light_declaration = {
    # TODO(human): 스키마 작성
}

In [ ]:
# 스키마로 Tool 생성 및 테스트
tools = types.Tool(function_declarations=[
    get_room_temperature_declaration,
    # set_light_declaration,  # 위에서 작성 후 주석 해제
])

config = types.GenerateContentConfig(
    tools=[tools],
    automatic_function_calling=types.AutomaticFunctionCallingConfig(disable=True)
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="거실 온도 알려줘",
    config=config
)

print("📞 함수 호출 요청:")
for fn in response.function_calls:
    print(f"  - {fn.name}({fn.args})")

---
## 3. 다중 함수 시스템 구축 (스마트홈)

스마트홈 제어를 위한 3개의 핵심 함수를 설계합니다:
1. `set_light` - 조명 제어
2. `set_thermostat` - 온도 조절기 제어
3. `lock_door` - 도어락 제어

In [ ]:
# 스마트홈 상태 저장소 (제공됨)
smart_home_state = {
    "lights": {
        "거실": {"brightness": 100, "color_temp": "daylight", "on": True},
        "안방": {"brightness": 50, "color_temp": "warm", "on": True},
        "주방": {"brightness": 80, "color_temp": "cool", "on": False},
    },
    "thermostat": {
        "temperature": 22,
        "mode": "auto",  # auto, heating, cooling, off
    },
    "doors": {
        "현관문": {"locked": True},
        "뒷문": {"locked": True},
        "차고문": {"locked": False},
    }
}

def get_smart_home_status():
    """현재 스마트홈 상태 출력"""
    print("🏠 스마트홈 현재 상태:")
    print(f"  💡 조명: {smart_home_state['lights']}")
    print(f"  🌡️  온도: {smart_home_state['thermostat']}")
    print(f"  🚪 도어: {smart_home_state['doors']}")

get_smart_home_status()

In [ ]:
# TODO(human): 3개의 스마트홈 제어 함수 구현
# 목적: 연관된 함수 세트 설계 연습
# 힌트: smart_home_state를 수정하고 결과 반환

def set_light(room: str, brightness: int, color_temp: str) -> dict:
    """방의 조명을 설정합니다.
    
    Args:
        room: 방 이름 (거실, 안방, 주방)
        brightness: 밝기 (0-100, 0은 끄기)
        color_temp: 색온도 (warm, cool, daylight)
    
    Returns:
        설정 결과
    """
    # TODO(human): 구현
    # 1. smart_home_state["lights"][room] 업데이트
    # 2. brightness가 0이면 on=False
    # 3. 결과 반환
    pass


def set_thermostat(temperature: int, mode: str) -> dict:
    """온도 조절기를 설정합니다.
    
    Args:
        temperature: 목표 온도 (16-30도)
        mode: 작동 모드 (auto, heating, cooling, off)
    
    Returns:
        설정 결과
    """
    # TODO(human): 구현
    pass


def lock_door(door_name: str, lock_state: bool) -> dict:
    """도어락을 제어합니다.
    
    Args:
        door_name: 문 이름 (현관문, 뒷문, 차고문)
        lock_state: 잠금 상태 (True=잠금, False=해제)
    
    Returns:
        제어 결과
    """
    # TODO(human): 구현
    pass

In [ ]:
# 스마트홈 함수 테스트 (함수 구현 후 실행)
# set_light("거실", 50, "warm")
# set_thermostat(24, "cooling")
# lock_door("차고문", True)
# get_smart_home_status()

In [ ]:
# 스마트홈 시스템 통합 테스트
SMART_HOME_FUNCTIONS = {
    "set_light": set_light,
    "set_thermostat": set_thermostat,
    "lock_door": lock_door,
    "get_room_temperature": get_room_temperature,
}

config = types.GenerateContentConfig(
    tools=list(SMART_HOME_FUNCTIONS.values()),
    system_instruction="당신은 스마트홈 AI 비서입니다. 사용자의 요청에 따라 조명, 온도, 도어락을 제어하세요."
)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents="집에 들어왔어. 거실 조명 켜고 온도 24도로 맞춰줘",
    config=config
)

print("🤖 AI 응답:")
print(response.text)
get_smart_home_status()

---
## 4. 순차 호출 파이프라인

함수 A의 결과를 함수 B의 입력으로 사용하는 체이닝 패턴입니다.

**예시 시나리오:**
1. `get_room_temperature("거실")` → 현재 온도 22도
2. 온도가 낮으면 → `set_thermostat(24, "heating")`

In [ ]:
# 순차 호출 예시 (제공됨)
def check_and_adjust_temperature(room: str, target_temp: int) -> dict:
    """방 온도를 확인하고 필요시 조절합니다.
    
    Args:
        room: 확인할 방
        target_temp: 목표 온도
    
    Returns:
        조치 결과
    """
    # 1단계: 현재 온도 확인
    current = get_room_temperature(room)
    current_temp = current["temp"]
    
    # 2단계: 필요시 조절
    if current_temp < target_temp:
        action = "heating"
    elif current_temp > target_temp:
        action = "cooling"
    else:
        return {"action": "none", "message": f"{room} 이미 {target_temp}도입니다."}
    
    # set_thermostat이 구현되어 있다면 호출
    # result = set_thermostat(target_temp, action)
    
    return {
        "room": room,
        "current_temp": current_temp,
        "target_temp": target_temp,
        "action": action,
        "message": f"{room} {current_temp}도 → {target_temp}도 ({action})"
    }

# 테스트
result = check_and_adjust_temperature("안방", 24)
print(result)

In [ ]:
# TODO(human): 외출 모드 함수 구현
# 목적: 여러 함수를 순차적으로 호출하는 파이프라인 설계
# 힌트:
#   1. 모든 조명 끄기 (set_light로 각 방 brightness=0)
#   2. 온도 조절기 끄기 (set_thermostat mode="off")
#   3. 모든 문 잠그기 (lock_door로 각 문 lock_state=True)
# 예상: "외출 모드로 설정해줘" 요청에 모든 설정 일괄 처리

def activate_away_mode() -> dict:
    """외출 모드를 활성화합니다. 모든 조명을 끄고, 온도 조절기를 끄고, 모든 문을 잠급니다.
    
    Returns:
        외출 모드 설정 결과
    """
    # TODO(human): 구현
    # 1. 모든 조명 끄기
    # 2. 온도 조절기 끄기
    # 3. 모든 문 잠그기
    # 4. 결과 요약 반환
    pass

---
## 5. 종합 프로젝트: 스마트홈 에이전트

In [ ]:
# TODO(human): 스마트홈 AI 에이전트 완성
# 목적: Function Calling을 활용한 완전한 에이전트 구현
# 힌트:
#   1. 모든 스마트홈 함수 등록 (set_light, set_thermostat, lock_door, get_room_temperature, activate_away_mode)
#   2. 자동 함수 호출 모드 사용
#   3. 대화 기록 유지로 컨텍스트 유지
# 예상: 자연어로 스마트홈 전체 제어 가능

def smart_home_agent():
    """스마트홈 AI 에이전트"""
    
    # 사용 가능한 함수 목록
    available_functions = [
        get_room_temperature,
        # TODO: 나머지 함수들 추가
    ]
    
    config = types.GenerateContentConfig(
        tools=available_functions,
        system_instruction="""당신은 스마트홈 AI 비서 '홈이'입니다.
        사용자의 요청에 따라 조명, 온도, 도어락을 제어합니다.
        현재 상태를 확인하고 적절한 조치를 취하세요.
        친근하고 도움이 되는 어조로 응답하세요."""
    )
    
    contents = []  # 대화 기록
    
    print("🏠 스마트홈 AI '홈이'입니다. 'quit'으로 종료합니다.")
    print("   예시: '거실 조명 50%로', '외출 모드', '안방 온도 알려줘'")
    print()
    
    # TODO: while 루프로 대화 구현
    # while True:
    #     user_input = input("You: ")
    #     if user_input.lower() == "quit":
    #         break
    #     ...
    #     print(f"\n홈이: {response.text}")
    #     get_smart_home_status()

# smart_home_agent()  # 실행하려면 주석 해제

---
## 6. 정리

### 핵심 요약

| 단계 | 내용 |
|-----|------|
| 1. 함수 정의 | 명확한 docstring과 타입 힌트 |
| 2. 스키마 작성 | FunctionDeclaration JSON 구조 |
| 3. 다중 함수 | 연관된 함수 세트 설계 |
| 4. 파이프라인 | 순차 호출로 복잡한 작업 처리 |
| 5. 에이전트 | 대화형 인터페이스 통합 |

### 학습 완료!

이제 다른 SDK(Claude, OpenAI)의 Function Calling도 유사한 패턴으로 학습할 수 있습니다.  
핵심은 **함수 정의 → 스키마 → 호출 처리**의 흐름입니다.